In [80]:
test = """        ...#
        .#..
        #...
        ....
...#.......#
........#...
..#....#....
..........#.
        ...#....
        .....#..
        .#......
        ......#.

10R5L5R10L4R5L5"""
source = test

offsets = [(2,0), (0,1), (1,1), (2,1), (2,2), (3,2)]
neighbors = [
    [
        [5,2], #done
        [1,2], #done
        [2,1],
        [3,0] # done
    ],
    [
        [2,0], #done
        [0,2], #done
        [5,1], #
        [4,2]
    ],
    [
        [3,0], #done
        [0,3], #
        [1,0], #done
        [4,1] #
    ],
    [
        [5,3], #
        [0,0], #done
        [2,0], #done
        [4,0] #done

    ], [
        [5,0], #done
        [3,0], #done
        [2,3], #
        [1,2] #done
    ], [
        [0,2], # done
        [1,3], #
        [4,0], #done
        [3,1],#
    ]
]

side = 4

In [81]:
useInput = True
if useInput == True:
    source = open("./day_22.txt").read()

    offsets = [(1,0), (2,0), (1,1), (0,2), (1,2), (0,3)]
    neighbors = [
        [
            [1,0], #Done
            [5,3], #x
            [3,2], # Done
            [2,0] #Done 
        ],
        [
            [4,2], #Done
            [5,0], #Done
            [0,0], # Done
            [2,3]
        ],
        [
            [1,1], #
            [0,0], # Done
            [3,1], #Done
            [4,0] # Done
        ],
        [
            [4,0], #Done
            [2,3], # Done
            [0,2], #Done
            [5,0] # Done

        ], [
            [1,2], # Done
            [2,0], # Done
            [3,0], # Done
            [5,3] #
        ], [
            [4,1], # 
            [3,0], # Done
            [0,1], # x
            [1,0],#Done
        ]
    ]
    side = 50

## Step 0: Parse the source data

In [82]:
(board, instructions) = source.split("\n\n")

In [83]:
# Parse instructions
steps = []
digits = []
for char in instructions:
    if char in "LR":
        if(len(digits)): steps.append({"type": "move", "steps": int("".join(digits))})
        # instructions = instructions.replace(char, f" {char} ")
        steps.append({"type": "turn", "direction": char})
        digits = []
    else: digits.append(char)
if(len(digits)): steps.append({"type": "move", "steps": int("".join(digits))})


In [84]:
WARP = ' '
WALL = ','
SPACE = "_"

In [85]:
board = board.replace("#", WALL)
boardRows = [WARP + line.replace(" ", WARP) + WARP for line in  board.splitlines()]
boardRows = [row.replace('.', SPACE) for row in boardRows]
maxlen = max([len(row) for row in boardRows])
boardRows = ["".join(WARP for _ in range(maxlen)), *[row + "".join(WARP for _ in range(maxlen - len(row))) for row in boardRows], "".join(WARP for _ in range(maxlen))]
boardGrid = [list(row) for row in boardRows]


In [86]:
facings = [">", "^", "<", "v"]
def viz_grid(boardGrid, position = [-1, -1], spacer = "  ", spos = None, rfacing = None, sfacing = None):
    newArr = boardGrid.copy()
    print("Visualizing with position", position,"and spos of ", spos)
    rchar = 'R' if rfacing == None else facings[rfacing % len(facings)]
    if position[0] > -1: newArr[position[1]][position[0]] = rchar
    schar = 'S' if sfacing == None else facings[sfacing % len(facings)]
    if(spos != None): newArr[spos[1]][spos[0]] = schar
    print("\n".join([spacer.join(row) for row in newArr]))

In [87]:
import numpy

boardArr = numpy.array(boardGrid)
startRow = 1
startCol =  boardArr[startRow].tolist().index(SPACE)

## Part 1: Wrapping Map

In [88]:
def runStep(step, facing, position, grid, stepIndex = 0, isDebug = False):
    if isDebug: print("Starting", step, facing, position, stepIndex)
    (col, row) = position
    if step["type"] == "turn":
        if step["direction"] == "R":
            facing = (facing - 1) % 4
            grid = numpy.rot90(grid, 1, axes=(0, 1))
            c,r = position
            row = len(grid) - c-1
            col = r
        else:
            facing = (facing + 1) % 4
            grid = numpy.rot90(grid, 3, axes=(0, 1))
            c,r = position
            col = len(grid[0]) - r-1
            row = c
    else:
        steps = step["steps"]
        c = col
        r = grid[row]
        r2 = r.copy()
        r2[col] = "R"
        if isDebug: print(''.join(r2))
        l = len([c for c in r if c != WARP])
        offset = next(i for i in range(len(r)) if r[i] != WARP)
        wallpos = next(i for i in range(col, len(r)) if r[i] == WALL) if WALL in r[col:] else -1
        prevwall = next(i for i in range(offset, col) if r[i] == WALL) if col > offset and WALL in r[:col] else -1
        if wallpos > -1 and steps > wallpos - col-1:
            col = wallpos - 1
        elif col + steps < (wallpos if wallpos > -1 else (offset + l)):
            col = (((col-offset) + steps) ) + offset
        elif (prevwall > -1) and (((l-(col-offset)) + (prevwall - offset))<= steps):
            if(prevwall > offset): col = prevwall -1
            else: col = offset +l -1
        else:
            col = (((col-offset) + steps) % l) + offset
        if isDebug: 
            r2[col] = "S"
            print(''.join(r2))
        r2[col] = "S"
        if(r[col] != SPACE): 
            print("BADMANMBADMANBADMAN", r[col], step, (c,r), stepIndex)
            return None

    return (facing, (col, row), grid)

In [89]:
position = (startCol, startRow)
grid = boardArr.copy()
facing = 0

In [90]:
for i, step in enumerate(steps): facing, position, grid = runStep(step, facing, position, grid, i)

In [91]:
originalFacing = facing
while facing > 0:
    print("Rotating because facing is ", facing)
    facing, position, grid = runStep({"type": "turn", "direction": 'R'}, facing, position, grid)

1000 * position[1] + 4 * position[0] + originalFacing

75388

## Part 2: Borg spaceship

In [92]:
position = (startCol, startRow)
grid = boardArr.copy()
facing = 0
origins = numpy.add(numpy.multiply(numpy.array(offsets), side),1).astype(int)
sides = [grid[origins[i][1]:origins[i][1]+side, origins[i][0]:origins[i][0]+side] for i in range(len(offsets))]

In [93]:
def rotateToFacing(facing, position, faceIndex):
    newFace = sides[faceIndex].copy()
    c,r = position
    for i in range((-facing) % 4):
        newFace = numpy.rot90(newFace, 1, axes=(0, 1))
        newr = len(newFace) - c - 1
        c = r
        r = newr
    newPos = (c,r)
    return newPos, newFace

def rotateFromFacing(facing, position, faceIndex):
    return rotateToFacing(facing * -1, position, faceIndex)

def getAbsolutePosition(faceIndex, position):
    npos = numpy.array(position)
    npos = numpy.add(npos, numpy.array(origins[faceIndex]))
    return npos

def getAbsolutePositionOnFace(faceIndex, position, facing):
    newpos, _ = rotateFromFacing(facing, position, faceIndex)
    abspos = getAbsolutePosition(faceIndex, newpos)
    return abspos

def viz_abspos(faceIndex, position, facing):
    abspos = getAbsolutePosition(faceIndex, position)
    viz_grid(grid, abspos, rfacing=facing, spacer="")


In [94]:
from time import sleep
from IPython.display import clear_output

l = len(sides[0])

def runStep2(step, facing, faceIndex, position, stepIndex = 0, isDebug = True):
    if isDebug: print("Starting", "step", step, "facing", facing, "faceIndex", faceIndex, "position", position, stepIndex)
    (col, row) = position
    if step["type"] == "turn":
        facing = (facing - 1) % 4 if step["direction"] == "R" else (facing + 1) % 4
        if isDebug: print("Returning from turn", "step", step, "final facing", facing, "faceIndex", faceIndex, "position", position, stepIndex)
        return facing, faceIndex, position
    else:
        steps = step["steps"]
        pos, face = rotateToFacing(facing, position, faceIndex)
        if isDebug: 
            print("position after rotation", pos, "but before it was", position)
            print("neighbor is", neighbors[faceIndex][facing], "from faceIndex", faceIndex, "and facing ", facing)
        _, neighbor = rotateToFacing(facing + neighbors[faceIndex][facing][1], position, neighbors[faceIndex][facing][0])
        c = pos[0]
        g2 = [list("".join(face[y]) + " " + "".join(neighbor[y])) for y in range(len(face))]
        col = pos[0]
        row = pos[1]
        r0 = row
        r = face[pos[1]]
        n = neighbor[pos[1]]
        wallpos = next(i for i in range(col, len(r)) if r[i] == WALL) if WALL in r[col:] else -1
        neighborWall = next(i for i in range(len(r)) if n[i] == WALL) if WALL in n else -1
        if wallpos > -1 and steps > wallpos - col-1:
            col = wallpos - 1
        elif col + steps < (wallpos if wallpos > -1 else l):
            col = col + steps
        elif (neighborWall > -1) and ((l-col) + neighborWall<= steps):
            if neighborWall == 0 : col = l-1
            else: col = l+ neighborWall -1
        else:
            col = col + steps
        if isDebug: 
            print("I moved to col", col)
            print("=== grid at step "+str(stepIndex)+"===")
            viz_grid(g2, pos, spacer="", spos=(min(col if col < l else col + 1, 2*l), pos[1]))
            print("=== done with grid ===")
        #wait, are we on a different facing?
        if(col > l-1):
            col = col-l
            if isDebug: print("Buidling new facing from current facing of ", facing , "and neighbor's facing", neighbors[faceIndex][facing], "based on faceindex of", faceIndex)
            fi = neighbors[faceIndex][facing][0];
            if isDebug: print("updating facing based on ",neighbors[faceIndex][facing][1], facing, "with faceindex of ", faceIndex)
            facing = neighbors[faceIndex][facing][1] + facing
            faceIndex = fi
            if(col > l-1):
                newStep = {"type": "move", "steps": col - l}
                return runStep2(newStep, facing, faceIndex, (l-1, row), stepIndex, isDebug)
    return (facing, faceIndex, rotateFromFacing(facing, [col, row], faceIndex)[0])

In [95]:
facing = 0
faceIndex = 0
face = sides[faceIndex].copy()
position = (0,0)
history = [(getAbsolutePosition(0, (0,0)),(0,0), 0, 0)]
for i, step in enumerate(steps): 
    facing, faceIndex, position = runStep2(step, facing, faceIndex, position, stepIndex=i ,isDebug=False)
    absposition = getAbsolutePosition(faceIndex, position)
    history.append((absposition, position, facing, faceIndex))

In [96]:
# descriptions = [str(step[0]) + ": " + facings[step[1][2] % len(facings)] + ": " + str(step[1][3]) + " - " + ",".join([str(step[1][0][1]-1), str(step[1][0][0]-1)]) for step in enumerate(history)]
# descriptions[4000]

In [97]:
npos = getAbsolutePosition(faceIndex, position)
1000 * npos[1] + 4* npos[0] + (-facing % len(facings))


182170